In [1]:
using Pkg;
Pkg.activate("..");
Pkg.instantiate();

  Activating project at `~/dev/uni/amo-team-project`


In [30]:
Pkg.add("Gurobi");
Pkg.build("Gurobi");

   Resolving package versions...


  No Changes to `~/dev/uni/amo-team-project/Project.toml`
  No Changes to `~/dev/uni/amo-team-project/Manifest.toml`


    Building Gurobi → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/82a44a86f4dc4fa4510c9d49b0a74d3d73914d5c/build.log`


Precompiling project...


  ✓ Gurobi
  1 dependency successfully precompiled in 5 seconds. 201 already precompiled.


In [2]:
# using HiGHS
# using JuMP
# using Ipopt
# using DataFrames
# using JSON
# using Plots
# using StatsPlots
# using StatsBase
# using BenchmarkTools
# using Distributions
# using Distances

In [3]:
include("../src/rwth_parser.jl")
network, loads = read_data("../data/Scenario_2013.xlsx")
print()

In [4]:
include("../src/loads.jl")
local_loads = remove_non_data_rows(filter_month(loads, 1))
print()

In [5]:
include("../src/scenarios/generation.jl")
attack_scenarios = generate_attack_scenarios(network, 10, 10000)
weather_scenarios = generate_weather_scenarios(10000)
print()

In [6]:
n_subscenarios = 2
include("../src/scenarios/reduction.jl")
reduced_attack_scenarios = reduce_binary_scenarios(attack_scenarios, n_subscenarios)
reduced_weather_scenarios = reduce_continous_scenarios(weather_scenarios, n_subscenarios)
reduced_load_scenarios = reduce_continous_scenarios(local_loads, n_subscenarios)
print()

In [7]:
include("../src/scenarios/assemble.jl")
scenarios = cartesian_scenarios(
	reduced_load_scenarios,
	reduced_attack_scenarios,
	reduced_weather_scenarios,
)
print()

In [8]:
scenarios = translate_weather_to_capacity(scenarios, network.generators)
print()

In [9]:
scenario_dict = convert_df_to_scenarios(scenarios)
print()

In [10]:
network.busses[:B1].incoming

Set{Symbol} with 1 element:
  :LFB3TB1

In [37]:
include("../src/model.jl")
dispatch_model = DispatchModel(network, scenario_dict, 5.0)
init_model!(dispatch_model)
print()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-19


In [38]:
solve!(dispatch_model)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1421 rows, 759 columns and 3489 nonzeros
Model fingerprint: 0x26f0ac36
Variable types: 519 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [9e+01, 2e+03]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 20157.773898
Presolve removed 881 rows and 344 columns
Presolve time: 0.01s
Presolved: 540 rows, 415 columns, 1627 nonzeros
Variable types: 382 continuous, 33 integer (33 binary)

Root relaxation: objective 0.000000e+00, 315 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0

OPTIMAL::TerminationStatusCode = 1

In [13]:
results_r = convert_jump_container_to_df(vars[:r])

UndefVarError: UndefVarError: vars not defined

In [14]:
p_results = reformat_2d_df(convert_jump_container_to_df(vars[:P]))

UndefVarError: UndefVarError: vars not defined

In [15]:
m = JuMP.Model(HiGHS.Optimizer)

# variables
## first-stage
@variable(m, r[b in keys(busses)], Bin) # reinforce bus

## second-stage
@variable(m, 0 <= P[g in keys(generators), s in keys(scenarios)]) # power generation
@variable(m, 0 <= L_shed[b in keys(busses), s in keys(scenarios)] <= busses[b][:load]) # load shedding

@variable(m, z[b in keys(busses), s in keys(scenarios)], Bin) # bus outage

@variable(m, F[l in keys(lines), s in keys(scenarios)]) # power flow
@variable(m, δ[b in keys(busses), s in keys(scenarios)]) # voltage angle

print()

UndefVarError: UndefVarError: busses not defined

In [16]:
# objective
# minimize expected load shedding over all scenarios
@objective(m, Min, sum(sum(L_shed[b, s] for b in keys(busses)) * 1/length(scenarios) for s in keys(scenarios)))
print()

MethodError: MethodError: no method matching keys(::DataFrame)
Closest candidates are:
  keys(!Matched::Union{Tables.AbstractColumns, Tables.AbstractRow}) at ~/.julia/packages/Tables/T7rHm/src/Tables.jl:189
  keys(!Matched::Missings.EachReplaceMissing) at ~/.julia/packages/Missings/TpCXO/src/Missings.jl:94
  keys(!Matched::Number) at number.jl:93
  ...

In [17]:
# constraints

## first-stage
### reinforcement budget
@constraint(m, 
	reinforcement_budget,
	  sum(r[b] * busses[b][:reinforcement_cost] for b in keys(busses)) 
	<= budget
)

## second-stage
### power balance
@constraint(m, 
	power_balance[b in keys(busses), s in keys(scenarios)],
	sum(P[g, s] for g in busses[b][:generators]) 
	+ sum(F[l, s] for l in busses[b][:incoming]) 
	- sum(F[l, s] for l in busses[b][:outgoing]) 
	+ L_shed[b, s] 
	- busses[b][:load]
	== 0
)
### line flow
@constraint(m, 
	line_flow[l in keys(lines), s in keys(scenarios)],
	F[l, s] == lines[l][:susceptance] * (δ[lines[l][:from], s] - δ[lines[l][:to], s])
)
### line flow under outage upper
@constraint(m,
	line_flow_under_outage_upper[b in keys(busses), l in [busses[b][:incoming]; busses[b][:outgoing]], s in keys(scenarios)],
	F[l, s] <= lines[l][:capacity] * z[b, s]
)
### line flow under outage lower
@constraint(m,
	line_flow_under_outage_lower[b in keys(busses), l in [busses[b][:incoming]; busses[b][:outgoing]], s in keys(scenarios)],
	-lines[l][:capacity] * z[b, s] <= F[l, s] 
)

### generator under outage lower
@constraint(m,
	generator_under_outage_lower[g in keys(generators), s in keys(scenarios)],
	-generators[g][:capacity] * z[generators[g][:bus], s] <= P[g, s] 
)
### generator under outage upper
@constraint(m,
	generator_under_outage_upper[g in keys(generators), s in keys(scenarios)],
	P[g, s] <= generators[g][:capacity] * z[generators[g][:bus], s]
)

### reference angle
@constraint(m,
	reference_angle[s in keys(scenarios)],
	δ[:B1, s] == 0
)
### attacked busses
@constraint(m,
	busses[s in keys(scenarios), b in keys(busses)],
	z[b, s] <= 1 - (b in scenarios[s][:attacked_busses] ? 1 : 0) + r[b]
)

UndefVarError: UndefVarError: busses not defined

In [18]:
print(m)

Feasibility
Subject to


In [19]:
optimize!(m)

Model   status      : Empty
HiGHS run time      :          0.00


# Results
## Reinforce results

In [20]:
"""
Returns a `DataFrame` with the values of the variables from the JuMP container `var`.
The column names of the `DataFrame` can be specified for the indexing columns in `dim_names`,
and the name of the data value column by a Symbol `value_col` e.g. :Value
"""
function convert_jump_container_to_df(
        var::JuMP.Containers.DenseAxisArray;
        dim_names::Vector{Symbol}=Vector{Symbol}(),
        value_col::Symbol=:Value
    )

    if isempty(var)
        return DataFrame()
    end

    if length(dim_names) == 0
        dim_names = [Symbol("dim$i") for i in 1:length(var.axes)]
    end

    if length(dim_names) != length(var.axes)
        throw(ArgumentError("Length of given name list does not fit the number of variable dimensions"))
    end

    tup_dim = (dim_names...,)

    # With a product over all axis sets of size M, form an Mx1 Array of all indices to the JuMP container `var`
    ind = reshape([collect(k[i] for i in 1:length(dim_names)) for k in Base.Iterators.product(var.axes...)],:,1)

    var_val  = value.(var)

    df = DataFrame([merge(NamedTuple{tup_dim}(ind[i]), NamedTuple{(value_col,)}(var_val[(ind[i]...,)...])) for i in 1:length(ind)])

    # sort by :dim1

    df = df[sortperm(df[!, Symbol("dim1")]), :]

    return df
end

convert_jump_container_to_df

In [21]:
function reformat_2d_df(df::DataFrame)
	# sort dim 1 and 2
	df = df[sortperm(df[!, :dim1]), :]
	df = df[sortperm(df[!, :dim2]), :]
	return unstack(df, :dim1, :dim2, :Value)
end

reformat_2d_df (generic function with 1 method)

In [22]:
results_r = convert_jump_container_to_df(r)

UndefVarError: UndefVarError: r not defined

In [23]:
p_results = reformat_2d_df(convert_jump_container_to_df(P))

UndefVarError: UndefVarError: P not defined

In [24]:
L_shed_results = reformat_2d_df(convert_jump_container_to_df(L_shed))

UndefVarError: UndefVarError: L_shed not defined

In [25]:
z_results = reformat_2d_df(convert_jump_container_to_df(z))

UndefVarError: UndefVarError: z not defined

In [26]:
F_results = reformat_2d_df(convert_jump_container_to_df(F))

UndefVarError: UndefVarError: F not defined

In [27]:
δ_results = reformat_2d_df(convert_jump_container_to_df(δ))

UndefVarError: UndefVarError: δ not defined

In [28]:
(reformat_2d_df(convert_jump_container_to_df(model)), F_results = reformat_2d_df(convert_jump_container_to_df(F))

ErrorException: syntax: incomplete: premature end of input